# Prepare

In [1]:
!pip install tqdm
!pip install -q datasets
# !pip install -q https://github.com/kpu/kenlm/archive/master.zip , no need as we do not use the perplexity calculation from the python module but it is always better to take a look :)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.6 MB/s eta 0:00:00


In [2]:
!apt -yq install\
  cmake\
  zlib1g-dev\
  libbz2-dev\
  liblzma-dev\
  libeigen3-dev\
  build-essential\
  libboost-test-dev\
  libboost-system-dev\
  libboost-thread-dev\
  libboost-program-options-dev

Reading package lists...
Building dependency tree...
Reading state information...
libboost-program-options-dev is already the newest version (1.71.0.0ubuntu2).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.71.0.0ubuntu2).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.71.0.0ubuntu2).
libboost-thread-dev set to manually installed.
libbz2-dev is already the newest version (1.0.8-2).
libbz2-dev set to manually installed.
libboost-test-dev is already the newest version (1.71.0.0ubuntu2).
libboost-test-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
liblzma-dev is already the newest version (5.2.4-1ubuntu1.1).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).
zlib1g-dev set to manually installed.
Suggested packag

In [3]:
%%shell
if [ ! -d kenlm ]; then
  wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
  mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
else
  echo kenlm already exist
fi

--2023-05-17 15:34:13--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K   512KB/s    in 0.9s    

2023-05-17 15:34:15 (512 KB/s) - written to stdout [491888/491888]

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gn

In [4]:
import os
import re
import psutil
import datasets
from tqdm.auto import tqdm

# Data Loading

In [6]:
# dataset = datasets.load_dataset("wikitext", "wikitext-2-v1")
dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1")
dataset

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [34]:
dataset['test']['text']

[' = Robert Boulter = \n',
 ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n',
 ' In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill . He appea

In [7]:
def mask_vowels(text, mask=""):
    text_with_no_vowels = re.sub(
        r"[AEIOU]",
        mask,
        text,
        flags=re.IGNORECASE,
    )
    return text_with_no_vowels

In [8]:
def prepare_example(example):
        example["processed_text"] = example["text"]
        example["consonants"] = mask_vowels(example["processed_text"])
        example["masked_consonants"] = mask_vowels(example["processed_text"], mask="a")
        return example

dataset["train"] = (
    dataset["train"]
    .filter(lambda example: len(example['text'])>0)
    .map(prepare_example)
)
dataset["validation"] = (
    dataset["validation"]
    .filter(lambda example: len(example['text'])>0)
    .map(prepare_example)
)
dataset["test"] = (
    dataset["test"]
    .filter(lambda example: len(example['text'])>0)
    .map(prepare_example)
)

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/23767 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

# Utility Functions

In [9]:
def estimate_memory_to_use_by_lm_modeler(verbose=True, margin=5):
  '''
  this funciton will estimate the memory to use by LM
  it calculates the current usage of memory adding some margin
  then return the remaining percentage of memory to be used
  '''
  # get the current memory used
  current_memory_usage = psutil.virtual_memory().percent
  # add some margin
  current_memory_usage += margin
  memory_to_use = int(100 - current_memory_usage)
  if verbose:
    print()
    print('#'*80)
    print('Estimating the LM model using:',f'{memory_to_use}% of memory.')
    print('#'*80)
    print()
  return memory_to_use

In [10]:
def train_lm_model(
    ngram:int,
    train_dataset:list,
    dataset_name:str,
    models_path='LMs',
    use_tqdm=True,
):
    !mkdir -p "{models_path}"/"{dataset_name}"/"{ngram}"
    
    print(" Writing train_dataset into file ".center(80, "#"))

    train_dataset = tqdm(train_dataset) if use_tqdm else train_dataset

    with open(
        f"{models_path}/{dataset_name}/train_dataset.txt",
        "w",
    ) as train_dataset_file:
        for item in train_dataset:
            train_dataset_file.write(item)
            train_dataset_file.write("\n")

    memory_to_use = estimate_memory_to_use_by_lm_modeler()

    !kenlm/build/bin/lmplz -o "{ngram}" -S "{memory_to_use}"% --discount_fallback < "{models_path}"/"{dataset_name}"/train_dataset.txt > "{models_path}"/"{dataset_name}"/"{ngram}"/lm.arpa

    print("#" * 80)
    print(" Converting .arpa file to binary file. ".center(80, "#"))

    !kenlm/build/bin/build_binary "{models_path}"/"{dataset_name}"/"{ngram}"/lm.arpa "{models_path}"/"{dataset_name}"/"{ngram}"/lm.binary

In [11]:
def get_perplexity_and_OOVs(
    ngram:int,
    test_dataset:list,
    dataset_name:str,
    models_path='LMs',
    overwrite_files=False,
    print_to_console=True,
):
    
    with open(
        f"{models_path}/{dataset_name}/test_dataset.txt",
        "w",
    ) as test_dataset_file:
        test_dataset_file.write("\n".join(test_dataset))

    # calculate and dump to a file
    !kenlm/build/bin/query "{models_path}"/"{dataset_name}"/"{ngram}"/lm.binary < "{models_path}"/"{dataset_name}"/test_dataset.txt > "{models_path}"/"{dataset_name}"/results.txt

    with open(f"{models_path}/{dataset_name}/results.txt") as f:
        lines = f.read().splitlines()

    # collect
    perplexity_with_OOVs_line = lines[-4]
    perplexity_without_OOVs_line = lines[-3]
    counts_of_OOVs_line = lines[-2]

    perplexity_with_OOVs = float(perplexity_with_OOVs_line.split("Perplexity including OOVs:")[-1].strip())
    perplexity_without_OOVs = float(perplexity_without_OOVs_line.split("Perplexity excluding OOVs:")[-1].strip())
    counts_of_OOVs = int(counts_of_OOVs_line.split("OOVs:")[-1].strip())

    return perplexity_with_OOVs, perplexity_without_OOVs, counts_of_OOVs

In [12]:
results = {}

# Train and collect the results

## Normal Text

In [13]:
train_dataset = dataset['train']['processed_text']
len(train_dataset)

23767

In [14]:
train_dataset[:2]

[' = Valkyria Chronicles III = \n',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n']

In [15]:
test_dataset =dataset['test']['processed_text']
len(test_dataset)

2891

In [16]:
test_dataset[:2]

[' = Robert Boulter = \n',
 ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n']

In [17]:
results['all-characters'] = {}

In [18]:
for ngram in range(2,7):
  train_lm_model(
      ngram=ngram,
      dataset_name='wikitext2',
      train_dataset=train_dataset,
  )
  perplexity_with_OOVs, perplexity_without_OOVs, counts_of_OOVs = get_perplexity_and_OOVs(
      ngram=ngram,
      test_dataset=test_dataset,
      dataset_name='wikitext2',
    )
  results['all-characters'][ngram] = {
      'perplexity_with_OOVs':perplexity_with_OOVs,
      'perplexity_without_OOVs':perplexity_without_OOVs,
      'counts_of_OOVs':counts_of_OOVs,
  }

####################### Writing train_dataset into file ########################


  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 87% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 76619
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:919428 2:11842674688
Statistics:
1 76619 D1=0.606829 D2=1.06583 D3+=1.47272
2 705257 D1=0.733467 D2=1.13519 D3+=1.42715
Memory estimate for binary LM:
type       kB
probing 14342 assuming -p 1.5
probing 14641 assuming -r models -p 1.5
trie     5928 without quantization
trie     3949 assuming -q 8 -b 8 quantization 
trie     5928 assuming -a 22 array pointer compression
trie     3949 assuming -

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 76619
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:919428 2:4071840512 3:7634700800
Statistics:
1 76619 D1=0.606829 D2=1.06583 D3+=1.47272
2 705257 D1=0.763577 D2=1.15175 D3+=1.42143
3 1463279 D1=0.851022 D2=1.25646 D3+=1.41133
Memory estimate for binary LM:
type       kB
probing 44196 assuming -p 1.5
probing 48628 assuming -r models -p 1.5
trie    19064 without quantization
trie    10913 assuming -q 8 -b 8 quantization 
trie    17948 assuming

...


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 76619
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:919428 2:1992602624 3:3736130048 4:5977808384
Statistics:
1 76619 D1=0.606829 D2=1.06583 D3+=1.47272
2 705257 D1=0.763577 D2=1.15175 D3+=1.42143
3 1463279 D1=0.879643 D2=1.27237 D3+=1.44934
4 1822384 D1=0.925531 D2=1.41189 D3+=1.44963
Memory estimate for binary LM:
type    MB
probing 82 assuming -p 1.5
probing 95 assuming -r models -p 1.5
trie    38 without quantization
trie    21 assuming -q 

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 76619
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:919428 2:1142101504 3:2141440512 4:3426304512 5:4996694528
Statistics:
1 76619 D1=0.606829 D2=1.06583 D3+=1.47272
2 705257 D1=0.763577 D2=1.15175 D3+=1.42143
3 1463279 D1=0.879643 D2=1.27237 D3+=1.44934
4 1822384 D1=0.948087 D2=1.42476 D3+=1.53324
5 1924288 D1=0.961801 D2=1.54834 D3+=1.50361
Memory estimate for binary LM:
type     MB
probing 126 assuming -p 1.5
probing 149 assuming -r models -

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 76619
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:919428 2:720402496 3:1350754688 4:2161207552 5:3151761152 6:4322415104
Statistics:
1 76619 D1=0.606829 D2=1.06583 D3+=1.47272
2 705257 D1=0.763577 D2=1.15175 D3+=1.42143
3 1463279 D1=0.879643 D2=1.27237 D3+=1.44934
4 1822384 D1=0.948087 D2=1.42476 D3+=1.53324
5 1924288 D1=0.978668 D2=1.54596 D3+=1.67349
6 1941253 D1=0.977626 D2=1.65528 D3+=1.46921
Memory estimate for binary LM:
type     MB
pro

In [19]:
results['all-characters']

{2: {'perplexity_with_OOVs': 515.8875627657986,
  'perplexity_without_OOVs': 361.67884559681767,
  'counts_of_OOVs': 9864},
 3: {'perplexity_with_OOVs': 444.0220148837052,
  'perplexity_without_OOVs': 309.1449165199793,
  'counts_of_OOVs': 9864},
 4: {'perplexity_with_OOVs': 434.91683171463256,
  'perplexity_without_OOVs': 302.7455047662003,
  'counts_of_OOVs': 9864},
 5: {'perplexity_with_OOVs': 433.2175015415488,
  'perplexity_without_OOVs': 301.60320641418133,
  'counts_of_OOVs': 9864},
 6: {'perplexity_with_OOVs': 432.9044353510737,
  'perplexity_without_OOVs': 301.405915826675,
  'counts_of_OOVs': 9864}}

## Consonants

In [20]:
train_dataset = dataset['train']['consonants']
len(train_dataset)

23767

In [21]:
train_dataset[:2]

[' = Vlkyr Chrncls  = \n',
 ' Snjō n Vlkyr 3 : nrcrdd Chrncls ( Jpns : 戦場のヴァルキュリア3 , lt . Vlkyr f th Bttlfld 3 ) , cmmnly rfrrd t s Vlkyr Chrncls  tsd Jpn , s  tctcl rl @-@ plyng vd gm dvlpd by Sg nd Md.Vsn fr th PlySttn Prtbl . Rlsd n Jnry 2011 n Jpn , t s th thrd gm n th Vlkyr srs . mplyng th sm fsn f tctcl nd rl @-@ tm gmply s ts prdcssrs , th stry rns prlll t th frst gm nd fllws th " Nmlss " ,  pnl mltry nt srvng th ntn f Gll drng th Scnd rpn Wr wh prfrm scrt blck prtns nd r pttd gnst th mprl nt " Clmty Rvn " . \n']

In [22]:
test_dataset = dataset['test']['consonants']
len(test_dataset)

2891

In [23]:
results['consonants'] = {}

In [24]:
for ngram in range(2,7):
  train_lm_model(
      ngram=ngram,
      dataset_name='wikitext2',
      train_dataset=train_dataset,
  )
  perplexity_with_OOVs, perplexity_without_OOVs, counts_of_OOVs = get_perplexity_and_OOVs(
      ngram=ngram,
      test_dataset=test_dataset,
      dataset_name='wikitext2',
    )
  results['consonants'][ngram] = {
      'perplexity_with_OOVs':perplexity_with_OOVs,
      'perplexity_without_OOVs':perplexity_without_OOVs,
      'counts_of_OOVs':counts_of_OOVs,
  }

####################### Writing train_dataset into file ########################


  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2011985 types 56198
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:674376 2:11706785792
Statistics:
1 56198 D1=0.589534 D2=1.05969 D3+=1.48249
2 640988 D1=0.725819 D2=1.12468 D3+=1.4096
Memory estimate for binary LM:
type       kB
probing 12694 assuming -p 1.5
probing 12913 assuming -r models -p 1.5
trie     4994 without quantization
trie     3196 assuming -q 8 -b 8 quantization 
trie     4994 assuming -a 22 array pointer compression
trie     3196 assuming -a

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2011985 types 56198
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:674376 2:4071925504 3:7634860544
Statistics:
1 56198 D1=0.589534 D2=1.05969 D3+=1.48249
2 640988 D1=0.752834 D2=1.13374 D3+=1.41258
3 1414477 D1=0.849594 D2=1.24873 D3+=1.39648
Memory estimate for binary LM:
type       kB
probing 41313 assuming -p 1.5
probing 45289 assuming -r models -p 1.5
trie    17257 without quantization
trie     9611 assuming -q 8 -b 8 quantization 
trie    16247 assuming

...


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2011985 types 56198
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:674376 2:1992644352 3:3736208128 4:5977933312
Statistics:
1 56198 D1=0.589534 D2=1.05969 D3+=1.48249
2 640988 D1=0.752834 D2=1.13374 D3+=1.41258
3 1414477 D1=0.876812 D2=1.26107 D3+=1.43273
4 1783858 D1=0.925448 D2=1.40584 D3+=1.43805
Memory estimate for binary LM:
type    MB
probing 79 assuming -p 1.5
probing 91 assuming -r models -p 1.5
trie    35 without quantization
trie    19 assuming -q 

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2011985 types 56198
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:674376 2:1142125440 3:2141485312 4:3426376192 5:4996798976
Statistics:
1 56198 D1=0.589534 D2=1.05969 D3+=1.48249
2 640988 D1=0.752834 D2=1.13374 D3+=1.41258
3 1414477 D1=0.876812 D2=1.26107 D3+=1.43273
4 1783858 D1=0.947525 D2=1.41062 D3+=1.5271
5 1885960 D1=0.962023 D2=1.55142 D3+=1.47945
Memory estimate for binary LM:
type     MB
probing 121 assuming -p 1.5
probing 143 assuming -r models -p

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2011985 types 56198
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:674376 2:720417600 3:1350782976 4:2161252864 5:3151827200 6:4322505728
Statistics:
1 56198 D1=0.589534 D2=1.05969 D3+=1.48249
2 640988 D1=0.752834 D2=1.13374 D3+=1.41258
3 1414477 D1=0.876812 D2=1.26107 D3+=1.43273
4 1783858 D1=0.947525 D2=1.41062 D3+=1.5271
5 1885960 D1=0.97864 D2=1.54876 D3+=1.62739
6 1902371 D1=0.977891 D2=1.65632 D3+=1.48574
Memory estimate for binary LM:
type     MB
probi

In [25]:
results['consonants']

{2: {'perplexity_with_OOVs': 451.7376944077861,
  'perplexity_without_OOVs': 360.134587439442,
  'counts_of_OOVs': 6167},
 3: {'perplexity_with_OOVs': 388.0765053598678,
  'perplexity_without_OOVs': 308.0281862521026,
  'counts_of_OOVs': 6167},
 4: {'perplexity_with_OOVs': 379.1634416940041,
  'perplexity_without_OOVs': 300.88669366610276,
  'counts_of_OOVs': 6167},
 5: {'perplexity_with_OOVs': 377.4887962093219,
  'perplexity_without_OOVs': 299.58521230653014,
  'counts_of_OOVs': 6167},
 6: {'perplexity_with_OOVs': 377.2134917212116,
  'perplexity_without_OOVs': 299.3781567627533,
  'counts_of_OOVs': 6167}}

## Masked Consonants

In [26]:
train_dataset = dataset['train']['masked_consonants']
len(train_dataset)

23767

In [27]:
train_dataset[:2]

[' = Valkyraa Chranaclas aaa = \n',
 ' Sanjō na Valkyraa 3 : anracardad Chranaclas ( Japanasa : 戦場のヴァルキュリア3 , lat . Valkyraa af tha Battlafaald 3 ) , cammanly rafarrad ta as Valkyraa Chranaclas aaa aatsada Japan , as a tactacal rala @-@ playang vadaa gama davalapad by Saga and Madaa.Vasaan far tha PlayStataan Partabla . Ralaasad an Janaary 2011 an Japan , at as tha thard gama an tha Valkyraa saraas . amplayang tha sama fasaan af tactacal and raal @-@ tama gamaplay as ats pradacassars , tha stary rans parallal ta tha farst gama and fallaws tha " Namalass " , a panal malatary anat sarvang tha nataan af Gallaa darang tha Sacand aarapan War wha parfarm sacrat black aparataans and ara pattad agaanst tha amparaal anat " Calamaty Ravan " . \n']

In [28]:
test_dataset = dataset['test']['masked_consonants']
len(test_dataset)

2891

In [29]:
results['masked_consonants'] = {}

In [30]:
for ngram in range(2,7):
  train_lm_model(
      ngram=ngram,
      dataset_name='wikitext2',
      train_dataset=train_dataset,
  )
  perplexity_with_OOVs, perplexity_without_OOVs, counts_of_OOVs = get_perplexity_and_OOVs(
      ngram=ngram,
      test_dataset=test_dataset,
      dataset_name='wikitext2',
    )
  results['masked_consonants'][ngram] = {
      'perplexity_with_OOVs':perplexity_with_OOVs,
      'perplexity_without_OOVs':perplexity_without_OOVs,
      'counts_of_OOVs':counts_of_OOVs,
  }

####################### Writing train_dataset into file ########################


  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 67968
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:815616 2:11706645504
Statistics:
1 67968 D1=0.595952 D2=1.07232 D3+=1.49848
2 677640 D1=0.729484 D2=1.12914 D3+=1.428
Memory estimate for binary LM:
type       kB
probing 13637 assuming -p 1.5
probing 13902 assuming -r models -p 1.5
trie     5563 without quantization
trie     3662 assuming -q 8 -b 8 quantization 
trie     5563 assuming -a 22 array pointer compression
trie     3662 assuming -a 

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 67968
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:815616 2:4071876864 3:7634768896
Statistics:
1 67968 D1=0.595952 D2=1.07232 D3+=1.49848
2 677640 D1=0.759005 D2=1.13941 D3+=1.42926
3 1443877 D1=0.848652 D2=1.25235 D3+=1.40432
Memory estimate for binary LM:
type       kB
probing 42988 assuming -p 1.5
probing 47224 assuming -r models -p 1.5
trie    18407 without quantization
trie    10469 assuming -q 8 -b 8 quantization 
trie    17338 assuming

...


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 67968
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:815616 2:1992620416 3:3736163328 4:5977861632
Statistics:
1 67968 D1=0.595952 D2=1.07232 D3+=1.49848
2 677640 D1=0.759005 D2=1.13941 D3+=1.42926
3 1443877 D1=0.877155 D2=1.26216 D3+=1.4412
4 1815611 D1=0.924141 D2=1.40799 D3+=1.43665
Memory estimate for binary LM:
type    MB
probing 81 assuming -p 1.5
probing 93 assuming -r models -p 1.5
trie    37 without quantization
trie    20 assuming -q 8

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 67968
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:815616 2:1142111744 3:2141459584 4:3426335232 5:4996739072
Statistics:
1 67968 D1=0.595952 D2=1.07232 D3+=1.49848
2 677640 D1=0.759005 D2=1.13941 D3+=1.42926
3 1443877 D1=0.877155 D2=1.26216 D3+=1.4412
4 1815611 D1=0.946874 D2=1.41063 D3+=1.52688
5 1922228 D1=0.961096 D2=1.54589 D3+=1.50346
Memory estimate for binary LM:
type     MB
probing 124 assuming -p 1.5
probing 147 assuming -r models -p

  0%|          | 0/23767 [00:00<?, ?it/s]


################################################################################
Estimating the LM model using: 86% of memory.
################################################################################

=== 1/5 Counting and sorting n-grams ===
Reading /content/LMs/wikitext2/train_dataset.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 2051910 types 67968
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:815616 2:720408960 3:1350766848 4:2161227008 5:3151789312 6:4322454016
Statistics:
1 67968 D1=0.595952 D2=1.07232 D3+=1.49848
2 677640 D1=0.759005 D2=1.13941 D3+=1.42926
3 1443877 D1=0.877155 D2=1.26216 D3+=1.4412
4 1815611 D1=0.946874 D2=1.41063 D3+=1.52688
5 1922228 D1=0.978107 D2=1.54124 D3+=1.67147
6 1940541 D1=0.977309 D2=1.65417 D3+=1.4751
Memory estimate for binary LM:
type     MB
probi

In [31]:
results['masked_consonants']

{2: {'perplexity_with_OOVs': 481.18217577080964,
  'perplexity_without_OOVs': 358.59304899036135,
  'counts_of_OOVs': 8157},
 3: {'perplexity_with_OOVs': 411.8408691285545,
  'perplexity_without_OOVs': 305.0902002094905,
  'counts_of_OOVs': 8157},
 4: {'perplexity_with_OOVs': 402.7560905709988,
  'perplexity_without_OOVs': 298.28582748476975,
  'counts_of_OOVs': 8157},
 5: {'perplexity_with_OOVs': 401.0129790968865,
  'perplexity_without_OOVs': 297.02947543036936,
  'counts_of_OOVs': 8157},
 6: {'perplexity_with_OOVs': 400.73118476087024,
  'perplexity_without_OOVs': 296.83786687393183,
  'counts_of_OOVs': 8157}}

# Results

In [32]:
results

{'all-characters': {2: {'perplexity_with_OOVs': 515.8875627657986,
   'perplexity_without_OOVs': 361.67884559681767,
   'counts_of_OOVs': 9864},
  3: {'perplexity_with_OOVs': 444.0220148837052,
   'perplexity_without_OOVs': 309.1449165199793,
   'counts_of_OOVs': 9864},
  4: {'perplexity_with_OOVs': 434.91683171463256,
   'perplexity_without_OOVs': 302.7455047662003,
   'counts_of_OOVs': 9864},
  5: {'perplexity_with_OOVs': 433.2175015415488,
   'perplexity_without_OOVs': 301.60320641418133,
   'counts_of_OOVs': 9864},
  6: {'perplexity_with_OOVs': 432.9044353510737,
   'perplexity_without_OOVs': 301.405915826675,
   'counts_of_OOVs': 9864}},
 'consonants': {2: {'perplexity_with_OOVs': 451.7376944077861,
   'perplexity_without_OOVs': 360.134587439442,
   'counts_of_OOVs': 6167},
  3: {'perplexity_with_OOVs': 388.0765053598678,
   'perplexity_without_OOVs': 308.0281862521026,
   'counts_of_OOVs': 6167},
  4: {'perplexity_with_OOVs': 379.1634416940041,
   'perplexity_without_OOVs': 300.8